In [1]:
import tushare as ts
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import pandas as pd
import numpy as np
import baostock as bao
from baostock.data.resultset import ResultData
import baostock.common.contants as bao_consts
from pandas import Timestamp
from sz.stock_data.toolbox.data_provider import ts_code
from datetime import date, datetime, timedelta

ts.set_token('f96b1eeee9c8fddd357f2299cdedc1c88b2bb2a30ae1f772cf810dea')
ts_api = ts.pro_api()

blg = bao.login()
print('baostock login => error_code: %s error_msg: %s\n' %
      (blg.error_code, blg.error_msg))

from sz.stock_data.stock_data import StockData
StockData().setup(data_dir = '/Volumes/USBDATA/stock_data')


login success!
baostock login => error_code: 0 error_msg: success



/Users/kk/miniconda3/envs/quant/lib/python3.7/site-packages/tushare/util/upass.py:28: FutureWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated



In [29]:
df = ts_api.top_list(trade_date='20080104')
df.head(5)

,trade_date,ts_code,name,close,pct_change,turnover_rate,amount,l_sell,l_buy,l_amount,net_amount,net_rate,amount_rate,float_values,reason
0,20080104,000430.SZ,ST张家界,12.86,4.980,0.67,45760000.0,32236373.56,1.231668e+07,4.455305e+07,-1.991970e+07,-43.53,97.36,9.561140e+08,连续三个交易日收盘价达到涨幅限制价格的ST证券、*ST证券和未完成股改证券
1,20080104,000545.SZ,*ST吉药,10.37,-2.354,31.26,327550000.0,80331011.09,7.219430e+07,1.525253e+08,-8.136708e+06,-2.48,46.57,1.065004e+09,日换手率达到20%的前三只证券
2,20080104,000587.SZ,*ST光明,9.20,5.023,11.26,88550000.0,9110232.41,1.219064e+07,2.130087e+07,3.080404e+06,3.48,24.06,7.676375e+08,连续三个交易日收盘价达到涨幅限制价格的ST证券、*ST证券和未完成股改证券
3,20080104,000831.SZ,*ST关铝,33.44,10.000,7.77,609250000.0,23885356.62,1.338635e+08,1.577489e+08,1.099782e+08,18.05,25.89,8.061608e+09,日涨幅偏离值达到7%的前三只证券
4,20080104,000833.SZ,贵糖股份,11.89,9.991,16.42,268570000.0,22003370.09,4.828577e+07,7.028914e+07,2.628240e+07,9.79,26.17,1.687784e+09,日涨幅偏离值达到7%的前三只证券


In [33]:
df = ts_api.margin_detail(start_date='20140922', end_date='20140922')
df.head(5)

,trade_date,ts_code,rzye,rqye,rzmre,rqyl,rzche,rqchl,rqmcl,rzrqye
0,20140922,603993.SH,5.137719e+06,0.0,5137719.0,0.0,0.0,0.0,0.0,5.137719e+06
1,20140922,603000.SH,1.625911e+09,6351930.0,120795444.0,137250.0,124225286.0,83280.0,75280.0,1.632263e+09
2,20140922,601999.SH,1.130609e+07,0.0,11364262.0,0.0,58174.0,0.0,0.0,1.130609e+07
3,20140922,601998.SH,6.202393e+08,4749846.0,69104213.0,1072200.0,53872662.0,546400.0,978500.0,6.249892e+08
4,20140922,601996.SH,4.152161e+08,76386.0,51612105.0,8700.0,38641979.0,10300.0,4000.0,4.152925e+08


In [34]:
df.tail(5)

,trade_date,ts_code,rzye,rqye,rzmre,rqyl,rzche,rqchl,rqmcl,rzrqye
880,20140922,000012.SZ,6.401463e+08,6888814.0,53348484.0,910015.0,47408897.0,451100.0,488554.0,6.470351e+08
881,20140922,000009.SZ,2.457407e+09,7587934.0,145791412.0,588212.0,134985495.0,671472.0,572842.0,2.464995e+09
882,20140922,000006.SZ,5.413888e+08,850905.0,41961516.0,148500.0,33343889.0,73200.0,64000.0,5.422397e+08
883,20140922,000002.SZ,2.142585e+09,70999900.0,57814228.0,7700640.0,59015958.0,2738278.0,3200178.0,2.213585e+09
884,20140922,000001.SZ,2.298592e+09,22628965.0,143964922.0,2249400.0,100683535.0,1752527.0,1634827.0,2.321220e+09
